#### Attetion Mechanism

- "고양이가 잔다" -> "the cat sleeps"

- 입력 : "고양이가 잔다"

    - [전처리] -> \<start\> 고양이가 잔다 \<end\>  
    - [ENCODER] -> 각 단어마다 hidden state 생성  
    - [ATTENTION] -> 어디에 집중할지 계산  
    - [DECODER] -> 한 단어씩 생성 

- 출력 : the cat sleeps  

In [ ]:
# 원본
입력 = '고양이가 자요'
# 1단계 전처리
전처리_후 = f'<start> {input} <end>'
# 2단계 토큰화 (단어 -> 숫자)
단어_사전 = {
    '<pad>' : 0,        # 패딩
    '<start>' : 1,
    '<end>' : 2,
    '고양이가' : 3,
    '자요' : 4
}
# 최대길이 5개로 맞추자
정수_시퀀스 = [1,3,4,2,0]

<span style="color: Gold"> 1. EMBEDDING </span> (숫자 -> 백터: 숫자를 벡터로 바꾼다)

In [ ]:
# 임베딩레이어 === 256 차원
정수_시퀀스 = [1,3,4,2,0]
# Embedding...
임베딩_결과 = [
    [0.2, - 0.5, ..., 0.1],     # 정수_시퀀스 중 1번 숫자에 대한 것 -> (<start>)를 벡터화 한 것 (256차원)
    [0.2, - 0.5, ..., 0.3],     # 정수_시퀀스 중 3번 숫자에 대한 것 -> (고양이가)를 벡터화 한 것 (256차원)
    [0.2, - 0.5, ..., 0.4],     # 정수_시퀀스 중 4번 숫자에 대한 것 -> (자요)를 벡터화 한 것 (256차원)
    [0.2, - 0.5, ..., 0.2],     # 정수_시퀀스 중 2번 숫자에 대한 것 -> (<end>)를 벡터화 한 것 (256차원)
    [0.0, 0.0, ..., 0.0],     # 정수_시퀀스 중 0번 숫자에 대한 것 -> (<pad>)를 벡터화 한 것 (256차원)
]

<span style="color: Gold"> 2. ENCODER </span> (벡터->Hidden States : 벡터를 Hidden States로 만든다)

In [ ]:
# LSTM Encoder
# 각 타임스텝마다 hidden state 생성

# 초기상태
h0 = [0,0,0,...,0]  # 512차원 영벡터
c0 = [0,0,0,...,0]  # 512차원 영벡터

# ===========  타임스텝 1 : <start> 처리  ============
입력_t1 = [0.2, - 0.5, ..., 0.1]    # <start> 의 임베딩
h1, c1 = LSTM(입력_t1, h0,c0)       # h와 c는 이전의 것을 가져옴
# h1 = [0.1 .....] 512 차원

# ===========  타임스텝 2 : (고양이가) 처리  ============
입력_t2 = [0.2, - 0.5, ..., 0.3]
h2, c2 = LSTM(입력_t2, h1,c1)


# ===========  타임스텝 3 : (자요) 처리  ============
입력_t3 = [0.2, - 0.5, ..., 0.4]
h3, c3 = LSTM(입력_t3, h2,c2)


# ===========  타임스텝 4 : (<end>) 처리  ============
입력_t4 = [0.2, - 0.5, ..., 0.2]
h4, c4 = LSTM(입력_t4, h3,c3)


# ===========  타임스텝 5 : (<pad>)은 무시  ============
입력_t5 = [0.0, 0.0, ..., 0.0]
h5, c5 = LSTM(입력_t5, h4,c4)
# h5는 사용하지 않음(마스킹)

# ===========  인코더의 출력  ============
encoder_output = [h1, h2, h3, h4, h5] # 모든 hidden states
# 형태[1,5,512] ->  [배치, 시퀀스, units]
encoder_final_h = h4  # h5는 패딩 0이라 무시 -> hidden state(디코더 초기화용)
encoder_final_c = c4  # 패딩 무시 -> 마지막 cell state

# ============== 정리 =====================
<start> ----> h1
고양이가 ---> h2     "고양이" 정보를 얍축
자요    ----> h3    "자요" 정보를 압축
<end>   ---> h4     전체 문장 요약
<pad>  ----> h5     무시됨



<span style="color: lightblue;"> 인코더 요약 </span>  
LSTM은 hidden state과 cell state를 통해 문맥 정보를 누적해 학습한다  
첫번쨰 문장에서 인코딩된 마지막 \<end>를 다음 문장에서 초기값으로 설정하며 점차 문맥 정보를 누적한다

<span style="color: Gold"> 3. DECODER의 시작 (첫 단어 생성) </span> 

In [ ]:
# ============= 디코더 초기 상태 ===============
decoder_h = h4      # 인코더의 마지막 hidden state
decoder_c = c4

# ============= 첫번째 입력 : <start> 토큰 ===============
decoder_input = [1]  # <start> 의 인덱스
decoder_input_imbedding = [0.3, 0.1,...,0.2] # 256 차원 -->  <start>의 임베딩 값

<span style="color: Gold"> 4. Attention 계산 (첫 번째 생성) </span> 

- the 를 생성하기 위해서 어디를 봐야 하나? 가 관심사

In [ ]:
query = decoder_h  # 512 차원
# 의미 : 지금 뭘 번역하려고 하는가?

# key 준비
keys = [h1, h2, h3, h4]  # 패딩을 제외한 인코더 출력
# 의미 : 각 입력 단어의 정보

# value 준비(key와 동일)
values = [h1, h2, h3, h4]  # 패딩을 제외한 인코더 출력

# 1단계 score 계산
# W1, W2, V 는 학습된 가중치 행렬
# h1 <start>에 대한 score 계산
# h2 (고양이가) 에 대한 score 계산
# h3 (자요)에 대한 score 계산
# h4 <end>에 대한 score 계산
scores = [0.3, 0.8, 0.2, 0.1]

# 2단계 Attention Weights 계산
# softtmax 확률 분포    exp(scores)
exp_scores = [exp(0.3, exp(0.8)),...] = [1.35, 2.23, 1.22, 1.11]
# sum
sum_exp = 1.35+2.23+1.22+1.11 = 5.91
# 정규화
attention_weight = [
    1.35/5.91= 0.23,        # <start>
    2.23/5.91= 0.38,        # 고양이가      -> 가중치가 가장 높음
    1.22/5.91= 0.21,        # 자요
    1.11/5.91= 0.18,        # <end>

]

# 3단계 context vector 계산
# 가중합으로 인코더 정보 통합
context_vector = (
    0.23 x h1 +     <start>의정보 23%
    0.38 x h2 +     고양이가 의정보 38%  --> 가장 많이 참조
    0.21 x h3 +     자요 의정보 21%
    0.18 x h4 +     <end>의정보 18%
)

# 최종 결과 512차원
입력단어 별로 score을 구하고 attetion weight를 구해서 기여도를 본다
# 입력단어     score       Attention Weight    기여도
# ----         ----        --------            고양이가에 대한 기여도가 가장 높게 나오고 ---> 집중

<span style="color: Gold"> 5. DECODER 실행 (첫 단어 생성) </span> 

In [ ]:
# context vector 와 입력을 결합해서 단어 생성

# ---- 1단계 입력과 context 결합
decoder_input_emb = [0.3, ..., 0.2]  # <start>의 임베딩 (256차원)
context_vector = [0.32, ....., 0.22]

# concatenate 이어 붙이기
combined_input = [
    0.3, 0.2, ..., 0.2,     # 임베딩 256
    0.3, 0.2, ..., 0.2,     # context 512
]

# 형태 : 256 + 512  ->  768차원

# --- 2단계 LSTM 처리
decoder_outm, new_h, new_c = LSTM(
    combined_input,
    decoder_h,          # 이전 hidden state
    decoder_c           # 이전 cell state
)

# --- 3단계 : 단어 확률 분포 생성
# Dense Layer 사용 -> 512 + vocab_siz
logits = Dense(docoder_output)
# logits = [-0.5, 2.3, ...]     50개의 단어 점수

# softmax로 확률 변환
probailities = softmax(logits)
# probailities = [0.0, 0.42, 0.28, ...] 인덱스 1(the)가 가장 높게

# --- 4단계 가장 높은 확률의 단어 선택
predict_di = argmax(probailities)   # 1

# 단어 사전에서 lookup
영어_단어_사전 = {
    0 : '<pad>'         
    1 : 'the'  # -> 선택됨
    ...
}
predict_new = 'the'

In [ ]:
# ============= 디코더 초기 상태 ===============
decoder_h = new_h  
decoder_c = new_c

# ============= 두번째 입력 : <end> 토큰 ===============
decoder_input = [2]  # <end> 의 인덱스
decoder_input_imbedding = [0.3, 0.1,...,0.2] # 256 차원 -->  고양이가의 임베딩 값